In [1]:
import os
import base64
import requests
import random
import subprocess

from IPython.display import display

ENDPOINT_ID = "4160657552620650496"
PROJECT_ID = "vision-ml-first-test"

# This is cheating!!! Instead create an access key and store it in a key store!!!
GCLOUD_ACCESS_TOKEN = subprocess.check_output("gcloud auth print-access-token", shell=True).strip()

HEADERS = {'Authorization': "Bearer " + GCLOUD_ACCESS_TOKEN, 'Content-Type': 'application/json'}

END_POINT = "https://us-central1-prediction-aiplatform.googleapis.com/v1alpha1/projects/" + \
            PROJECT_ID + \
            "/locations/us-central1/endpoints/" + \
            ENDPOINT_ID + ":predict"

image_dir = "images"
IMAGE_COUNT = 10

REQ_TEMPLATE = '{ \
  "instances": [{ \
    "content": "REQUEST_DATA" \
  }], \
  "parameters": { \
    "confidenceThreshold": 0.5, \
    "maxPredictions": 5\
  }\
}'


In [2]:
# Find the files
files_all = os.listdir(image_dir)
files_sample = random.sample(files_all, IMAGE_COUNT)

for file_name in files_sample:
    
    # Get the image data
    full_file_name = image_dir + "/" + file_name
    in_file = open(full_file_name)
    file_data = in_file.read()
    in_file.close()
    
    # Encode and create request
    file_base64_string = base64 \
                .b64encode(file_data) \
                .decode('ascii')
    req_data = REQ_TEMPLATE.replace("REQUEST_DATA", file_base64_string)
    
    # Call the Google API
    resp = requests.post(END_POINT, data=req_data, headers=HEADERS)
    resp.raise_for_status()
    
    # Pull out the response data and print it
    prediction = resp.json()["predictions"][0]
    print file_name, " ", prediction["displayNames"][0], "Confidence =",prediction["confidences"][0]
    
    #Show the image
    display(Image(full_file_name))

HTTPError: 404 Client Error: Not Found for url: https://us-central1-prediction-aiplatform.googleapis.com/v1alpha1/projects/vision-ml-first-test/locations/us-central1/endpoints/4160657552620650496:predict